In [49]:
import pandas as pd
import numpy as np
from datetime import datetime
np.random.seed(43)

In [50]:
source_file = './WF_Rohdaten.xlsx'
current_calendar_file = './NBDF_schleppkalender_pull_2019-04-30.xlsx'
replies = './Rückmeldungne.xlsx'

In [51]:
results_file = f'Filled_calendar_{datetime.now().strftime("%Y-%m-%d")}.xlsx'

In [3]:
NBDF_member = pd.read_excel(source_file)
current_calendar = pd.read_excel(current_calendar_file)
replies_dict = {}
for sheet in pd.ExcelFile(replies).sheet_names:
    try:
        df = pd.read_excel(replies,sheet_name=sheet)
        first_name = df.columns[7]
        last_name = str(df[first_name][0])
        name = f'{first_name.strip().capitalize()[0]}. {last_name.strip().capitalize()}'
        unavaliable = list(df[(df['Kann ich auf keinen Fall']=='X') | (df['Kann ich auf keinen Fall']=='x') | (df['Kann ich auf keinen Fall']=='1')]['Datum'])
        replies_dict[name] = unavaliable
    except:
        pass

/home/jan/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [4]:
eingeteilte_WF = list(current_calendar['Windenfahrer'].dropna())
eingeteilte_helfer = list(current_calendar['Helfer'].dropna())

In [5]:
wf_dict = {}
for wf in eingeteilte_WF:
    if wf in wf_dict.keys():
        wf_dict[wf]+=1
    else:
        wf_dict[wf]= 1
        
helfer_dict = {}
for helfer in eingeteilte_helfer:
    if helfer in helfer_dict.keys():
        helfer_dict[helfer]+=1
    else:
        helfer_dict[helfer]=1

In [6]:
wf_dict

{'J. Macenka': 4,
 'M. Collischon': 1,
 'M. Stumpf': 1,
 'W. Huberth': 1,
 'G. Lechner': 2,
 'U. Görzen': 2,
 'K. Vogel': 1,
 'J. Schmilinsky': 1,
 'A. Hutzler': 2,
 'M. Schapler': 3,
 'G. Schmittlein': 1,
 'G. Grimm': 3,
 'M. Richter': 2,
 'W. Schaeper': 1,
 'J. Rottmann': 2}

In [7]:
helfer_dict

{'J. Macenka': 2,
 'J. Schilinsky': 1,
 'M. Uttenreuther': 2,
 'M. Lorenz': 2,
 'S. Trebeß': 2,
 'M. Langhammer': 2,
 'D. Bohl': 2,
 'J. Schrüffer': 1,
 'A. Greiner': 2,
 'M. Günther': 1,
 'D. Josef': 2,
 'B. Heinloth': 2,
 'L. Larrniczak': 2,
 'P. Teufel': 1,
 'T. Teufel': 1,
 'W. Weber': 1}

In [8]:
NBDF_member['Name_short'] = NBDF_member.apply(lambda row: row['Vorname'][0]+'. ' + row['Name'], axis=1)

In [9]:
NBDF_Helfer = NBDF_member[NBDF_member['WF HG'] != 'X']

In [10]:
Helfer_Namen = list(NBDF_Helfer['Name_short'])
Helfer_Schlepps = [int(s) for s in list(NBDF_Helfer['Anzahl Schlepps 2018'])]
total_schlepps = sum(Helfer_Schlepps)
Helfer_Prob = [s/total_schlepps for s in Helfer_Schlepps]

In [11]:
WF_GS = NBDF_member[['Name_short','Anzahl Schlepps 2018','WF HG']]
WF_GS = WF_GS[WF_GS['WF HG'].isin(['X'])]

In [12]:
WF_GS.sort_values(['Anzahl Schlepps 2018'],ascending=False)

,Name_short,Anzahl Schlepps 2018,WF HG
48,G. Schmittlein,55,X
25,E. Lehmann,33,X
42,J. Rottmann,27,X
10,G. Grimm,22,X
43,E. Ruhl,21,X
22,U. Knöchel,18,X
14,W. Huberth,17,X
6,M. Collischon,14,X
8,U. Görzen,14,X
23,U. Köck,14,X


In [13]:
verfügbare_wf = sorted(list(set(list(WF_GS['Name_short'])+eingeteilte_WF)))

In [14]:
verfügbare_wf

['A. Hutzler',
 'A. Meythaler',
 'E. Lehmann',
 'E. Pennig',
 'E. Ruhl',
 'G. Grimm',
 'G. Lechner',
 'G. Schmittlein',
 'J. Macenka',
 'J. Rottmann',
 'J. Schmilinsky',
 'K. Vogel',
 'M. Collischon',
 'M. Richter',
 'M. Schapler',
 'M. Stumpf',
 'U. Görzen',
 'U. Knöchel',
 'U. Köck',
 'W. Huberth',
 'W. Schaeper']

In [15]:
besetzte_liste = []
max_iter = 20
for wf in verfügbare_wf:
    if not wf in replies_dict.keys():
        replies_dict[wf] = []
dates = list(current_calendar['Datum'])
for idx, wf in enumerate(list(current_calendar['Windenfahrer'])):
    if str(wf) != 'nan':
        besetzte_liste.append(wf)
    else:
        geringster_wert = min(wf_dict.values())
        haben_genug = [key for key, value in wf_dict.items() if value > geringster_wert]
        if len(haben_genug) == 0:
            haben_genug = []
        n_iter = 0
        while True:
            n_iter += 1
            kandidat = np.random.choice(verfügbare_wf) 
            if n_iter > max_iter:
                besetzte_liste.append(kandidat)
                if kandidat in wf_dict.keys():
                    wf_dict[kandidat]+=1
                else:
                    wf_dict[kandidat]=1
                break
            if not kandidat in haben_genug:
                c_date = dates[idx]
                if not c_date in replies_dict[kandidat]:
                    besetzte_liste.append(kandidat)
                    if kandidat in wf_dict.keys():
                        wf_dict[kandidat]+=1
                    else:
                        wf_dict[kandidat]=1
                    break
current_calendar['Windenfahrer voll besetzt'] = besetzte_liste

In [16]:
besetzte_liste = []
max_iter = 30
for hf in Helfer_Namen:
    if not hf in replies_dict.keys():
        replies_dict[hf] = []
    if not hf in helfer_dict.keys():
        helfer_dict[hf] = 0
dates = list(current_calendar['Datum'])
for idx, wf in enumerate(list(current_calendar['Helfer'])):
    if str(wf) != 'nan':
        besetzte_liste.append(wf)
    else:
        geringster_wert = min(helfer_dict.values())
        haben_genug = [key for key, value in helfer_dict.items() if value > geringster_wert]
        if len(haben_genug) == 0:
            haben_genug = []
        n_iter = 0
        while True:
            n_iter += 1
            kandidat = np.random.choice(Helfer_Namen,p=Helfer_Prob)
            print(kandidat)
            if n_iter > max_iter:
                besetzte_liste.append(kandidat)
                if kandidat in wf_dict.keys():
                    helfer_dict[kandidat]+=1
                else:
                    helfer_dict[kandidat]=1
                break
            if not kandidat in haben_genug:
                c_date = dates[idx]
                if not c_date in replies_dict[kandidat]:
                    besetzte_liste.append(kandidat)
                    if kandidat in wf_dict.keys():
                        helfer_dict[kandidat]+=1
                    else:
                        helfer_dict[kandidat]=1
                    break
current_calendar['Helfer voll besetzt'] = besetzte_liste

A. Riedl
J. Siedentop
A. Riedl
B. Güth
J. Schrüffer
B. Güth
M. Stöcker
M. Günther
M. Berdami
J. Meyer
W. Weber
J. Siedentop
S. Rauh
B. Güth
J. Fischer
M. Stöcker
V. Volkmann
D. Bohl
J. Schmilinsky
M. Stöcker
H. Maag
W. Schaeper
P. Teufel
J. Schrüffer
M. Miunske
A. Greiner
A. Riedl
A. Riedl
B. Güth
J. Fischer
W. Schaeper
J. Meyer
J. Horn
M. Günther
T. Teufel
I. Miunske
T. Teufel
D. Bohl
M. Arnet
V. Volkmann
J. Schrüffer
G. Wünsch
A. Riedl
B. Güth
V. Volkmann
B. Güth
M. Miunske
V. Volkmann
M. Günther
A. Riedl
S. Tukscher
J. Meyer
V. Volkmann
G. Wünsch
A. Riedl
J. Meyer
W. Witt
G. Wünsch
A. Riedl
J. Fischer
A. Riedl
M. Stöcker
J. Schrüffer
J. Schrüffer
W. Weber
J. Schrüffer
M. Günther
J. Schmilinsky
J. Horn
M. Stöcker
I. Miunske
W. Schaeper
J. Siedentop
B. Güth
J. Schrüffer
B. Güth
T. Teufel
J. Meyer
W. Schaeper
T. Teufel
W. Weber
J. Meyer
M. Günther
V. Volkmann
W. Witt
V. Volkmann
S. Trebeß
W. Schaeper
A. Riedl
W. Schaeper
M. Günther
M. Uttenreuther
J. Schrüffer
W. Witt
J. Schwabel
I. Mi

In [17]:
current_calendar

,KW,Wochentag,Datum,Windenfahrer,Helfer,Windenfahrer voll besetzt,Helfer voll besetzt
0,14,Samstag,06.04.2019,J. Macenka,J. Macenka,J. Macenka,J. Macenka
1,14,Sonntag,07.04.2019,M. Collischon,J. Schilinsky,M. Collischon,J. Schilinsky
2,15,Samstag,13.04.2019,M. Stumpf,NaN,M. Stumpf,A. Riedl
3,15,Sonntag,14.04.2019,W. Huberth,M. Uttenreuther,W. Huberth,M. Uttenreuther
4,16,Karfreitag,19.04.2019,G. Lechner,M. Lorenz,G. Lechner,M. Lorenz
5,16,Samstag,20.04.2019,G. Lechner,NaN,G. Lechner,J. Siedentop
6,16,Sonntag,21.04.2019,U. Görzen,M. Uttenreuther,U. Görzen,M. Uttenreuther
7,17,Ostermontag,22.04.2019,NaN,NaN,E. Ruhl,B. Güth
8,17,Samstag,27.04.2019,J. Macenka,J. Macenka,J. Macenka,J. Macenka
9,17,Sonntag,28.04.2019,NaN,M. Lorenz,U. Knöchel,M. Lorenz


In [18]:
wf_dict

{'J. Macenka': 4,
 'M. Collischon': 3,
 'M. Stumpf': 3,
 'W. Huberth': 3,
 'G. Lechner': 3,
 'U. Görzen': 3,
 'K. Vogel': 3,
 'J. Schmilinsky': 3,
 'A. Hutzler': 3,
 'M. Schapler': 3,
 'G. Schmittlein': 3,
 'G. Grimm': 3,
 'M. Richter': 4,
 'W. Schaeper': 3,
 'J. Rottmann': 4,
 'E. Ruhl': 3,
 'U. Knöchel': 3,
 'E. Lehmann': 3,
 'E. Pennig': 3,
 'U. Köck': 3,
 'A. Meythaler': 4}

In [19]:
helfer_dict

{'J. Macenka': 2,
 'J. Schilinsky': 1,
 'M. Uttenreuther': 2,
 'M. Lorenz': 2,
 'S. Trebeß': 1,
 'M. Langhammer': 2,
 'D. Bohl': 2,
 'J. Schrüffer': 1,
 'A. Greiner': 2,
 'M. Günther': 1,
 'D. Josef': 2,
 'B. Heinloth': 2,
 'L. Larrniczak': 2,
 'P. Teufel': 1,
 'T. Teufel': 1,
 'W. Weber': 1,
 'M. Arnet': 1,
 'M. Berdami': 1,
 'S. Betzmeir': 1,
 'M. Boos': 1,
 'O. Clemens': 0,
 'J. Fischer': 1,
 'B. Güth': 1,
 'J. Horn': 1,
 'P. Hubertz': 1,
 'B. Igna': 0,
 'R. Jungkunz': 0,
 'H. Kappel': 0,
 'C. Knarr': 1,
 'W. Knarr': 1,
 'H. Maag': 1,
 'S. Meinlschmidt': 0,
 'J. Meyer': 1,
 'S. Mickley': 0,
 'I. Miunske': 1,
 'M. Miunske': 1,
 'A. Möllmann': 1,
 'C. Muehlhaus': 1,
 'R. Pabst': 1,
 'S. Rauh': 1,
 'A. Riedl': 1,
 'W. Schaeper': 3,
 'B. Schmidt': 0,
 'J. Schmilinsky': 1,
 'J. Schwabel': 1,
 'J. Siedentop': 1,
 'M. Stöcker': 1,
 'M. Stumpf': 1,
 'J. Tarach': 0,
 'S. Tukscher': 1,
 'V. Volkmann': 1,
 'W. Witt': 1,
 'G. Wünsch': 1}

In [44]:
neue_helfer = current_calendar[current_calendar['Helfer'].isnull()].sort_values('Helfer voll besetzt').reset_index()[['Datum','Helfer voll besetzt']]

In [46]:
neue_wf = current_calendar[current_calendar['Windenfahrer'].isnull()].sort_values('Windenfahrer voll besetzt').reset_index()[['Datum','Windenfahrer voll besetzt']]

In [47]:
neue_wf

,Datum,Windenfahrer voll besetzt
0,03.08.2019,A. Hutzler
1,27.07.2019,A. Meythaler
2,14.09.2019,A. Meythaler
3,08.06.2019,A. Meythaler
4,20.10.2019,A. Meythaler
5,28.09.2019,E. Lehmann
6,25.05.2019,E. Lehmann
7,10.06.2019,E. Lehmann
8,29.09.2019,E. Pennig
9,30.05.2019,E. Pennig


In [52]:
with pd.ExcelWriter(results_file) as writer:
    current_calendar.to_excel(writer, sheet_name='calendar',index=False)
    neue_wf.to_excel(writer, sheet_name='neue_wf')
    neue_helfer.to_excel(writer, sheet_name='neue_helfer')
print('All Done')

All Done
